In [ ]:
import os
from pathlib import Path

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HOME"] = str(Path.cwd().joinpath("cache"))

# Test Scheduler by Gaussian Mixture Model

In [ ]:
import itertools

import matplotlib.pyplot as plt
import torch

from gaussian import (
    GaussianModelScheduler as GaussianMixtureModelScheduler,
    GaussianModelPipeline as GaussianMixtureModelPipeline,
)
from gmm import GaussianMixtureModel
from sampler import SAMPLER_FORMULATION_TABLE

In [ ]:
def draw_inference_result(
    pipeline: GaussianMixtureModelPipeline,
    batch_size: int,
    num_inference_steps: int,
    seed: int
):
    plt.figure(figsize=(16, 22))

    algorithm_types = ["ode", "sde"]
    prediction_types = ["epsilon", "sample", "velocity"]
    timestep_schedules = ["linear_lognsr", "cosine_lognsr", "power_lognsr", "uniform"]

    for row_id, (algorithm_type, prediction_type) in enumerate(itertools.product(algorithm_types, prediction_types)):
        pipeline.scheduler.config.algorithm_type = algorithm_type
        pipeline.scheduler.config.prediction_type = prediction_type
        for col_id, timestep_schedule in enumerate(timestep_schedules):
            if algorithm_type == "sde" and prediction_type == "velocity":
                continue
            pipeline.scheduler.config.timestep_schedule = timestep_schedule
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)

            samples = pipeline(batch_size=batch_size, num_inference_steps=num_inference_steps)
            samples = samples.detach().cpu().numpy()
            mean, std = samples.mean(axis=0), samples.std(axis=0)
            print(mean, std)

            plt.subplot(len(algorithm_types) * len(prediction_types), len(timestep_schedules), row_id * len(timestep_schedules) + col_id + 1)
            plt.hist2d(samples[:, 0], samples[:, 1], bins=[256, 256], range=[[-3, 3], [-3, 3]], cmap="plasma", cmin=1)
            plt.title(f"{algorithm_type} {prediction_type} {timestep_schedule}")
            plt.axis("equal")

    plt.show()

In [ ]:
seed = 0
batch_size = 16384
num_inference_steps = 32

## Test Rectified Flow's Formulation

In [ ]:
FORMULATION = SAMPLER_FORMULATION_TABLE["Rectified Flow"]

pipeline = GaussianMixtureModelPipeline(
    GaussianMixtureModel(
        num_groups_per_model=8,
        num_gs_per_group=8,
        model_radius=2,
        group_radius=0.4,
        sigma=0.04
    ),
    GaussianMixtureModelScheduler(
        t_min=0.0001,
        t_max=0.9999,
        sigma_data=1.0,
        scale_fn=FORMULATION["scale_fn"],
        scale_deriv_fn=FORMULATION["scale_deriv_fn"],
        sigma_fn=FORMULATION["sigma_fn"],
        sigma_deriv_fn=FORMULATION["sigma_deriv_fn"],
        nsr_inv_fn=FORMULATION["nsr_inv_fn"],
        prediction_type="epsilon",
        algorithm_type="ode",
        timestep_schedule="cosine_lognsr"
    )
)
if torch.cuda.is_available():
    pipeline = pipeline.to(device=torch.device("cuda"))

In [ ]:
draw_inference_result(pipeline, batch_size, num_inference_steps, seed)

## Test EDM's Formulation

In [ ]:
FORMULATION = SAMPLER_FORMULATION_TABLE["EDM"]

pipeline.scheduler = GaussianMixtureModelScheduler(
    t_min=0.001,
    t_max=100.0,
    sigma_data=1.0,
    scale_fn=FORMULATION["scale_fn"],
    scale_deriv_fn=FORMULATION["scale_deriv_fn"],
    sigma_fn=FORMULATION["sigma_fn"],
    sigma_deriv_fn=FORMULATION["sigma_deriv_fn"],
    nsr_inv_fn=FORMULATION["nsr_inv_fn"],
    prediction_type="epsilon",
    algorithm_type="ode",
    timestep_schedule="cosine_lognsr"
)

In [ ]:
draw_inference_result(pipeline, batch_size, num_inference_steps, seed)

In [ ]:
plt.figure(figsize=(5, 5))

samples = torch.distributions.Normal(
    loc=pipeline.model.mu.detach().cpu(),
    scale=pipeline.model.sigma
).sample((batch_size // len(pipeline.model.mu.detach().cpu()),)).reshape(-1, 2).cpu().numpy()

plt.hist2d(samples[:, 0], samples[:, 1], bins=[256, 256], range=[[-3, 3], [-3, 3]], cmap="plasma", cmin=1)
plt.title("Ground Truth")

plt.show()